<a href="https://colab.research.google.com/github/khemsu/LangGraph_tut/blob/main/Recruitment_Agent_using_LangGraph_and_Neo4j_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install -q langchain langgraph openai neo4j pydantic


In [25]:
!pip install -U langchain langchain-openai


In [37]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from langchain_core.output_parsers import JsonOutputParser
from typing import TypedDict, List, Dict, Any
from neo4j import GraphDatabase
from pydantic import BaseModel

# === Neo4j Configuration ===
NEO4J_URI = "neo4j+s://c98ee5ba.databases.neo4j.io"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "zqa66tKG0g1xGgALB6gxU6ZGJxqnAQ3hnLUD3GfU6bc"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter OPENAI key here.")


# === OpenAI Configuration ===
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")


Enter OPENAI key here.··········


In [29]:
class Candidate(BaseModel):
    id: str
    name: str
    email: str
    phone: str
    location: str
    skills: List[str]
    experience: int


In [30]:
class GraphState(TypedDict):
    input: str
    results: List[Dict[str, Any]]


In [31]:
def get_all_candidates():
    query = "MATCH (c:Candidate) RETURN c"
    with driver.session() as session:
        result = session.run(query)
        candidates = []
        for record in result:
            c = record["c"]
            candidates.append(c)
        return candidates


In [32]:
prompt = ChatPromptTemplate.from_template("""
You are a helpful recruiter assistant. Filter the following candidates based on the query below.

Query: {input}

Candidates:
{candidates}

Return the list of matching candidates in JSON format with keys:
id, name, email, phone, location, skills, experience
""")

parser = JsonOutputParser(pydantic_object=List[Candidate])


In [33]:
def filter_candidates(state: GraphState):
    input_text = state["input"]
    candidates = get_all_candidates()

    formatted_candidates = "\n".join([str(dict(c)) for c in candidates])

    chain = prompt | llm | parser

    structured_result = chain.invoke({
        "input": input_text,
        "candidates": formatted_candidates
    })

    return {"results": structured_result, "input": input_text}


In [35]:
graph = StateGraph(GraphState)

graph.add_node("FilterCandidates", filter_candidates)
graph.set_entry_point("FilterCandidates")
graph.set_finish_point("FilterCandidates")

recruitment_agent = graph.compile()


In [38]:
query = "Find candidate who has experience on mongoDB"

response = recruitment_agent.invoke({"input": query})
for candidate in response["results"]:
    print(candidate)


{'id': 'C102', 'name': 'Biplav Ghale', 'email': 'BiplavGjale7.com', 'phone': '+977-9815623687', 'location': 'Kathmandu', 'skills': ['Python', 'FastAPI', 'MongoDB'], 'experience': 3}
